In [162]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or '3' to suppress all messages
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import random
label_encoder = LabelEncoder()
pd.options.mode.chained_assignment = None


In [163]:
label_path = './data/data_by_table/pre_24h_data_v6.csv'
flag_data_path = './data/data_by_table/ground_truth.csv'
raw_data_path = './data/data_by_table/pre_24_merged_30_rows_12_07.csv'
mode_data_path = './data/data_by_table/pre_24_merged_30_rows_12_07.csv'

data_df = pd.read_csv(mode_data_path)
flag_data_df = pd.read_csv(flag_data_path)
label_df = pd.read_csv(label_path)
mode_df = pd.read_csv(mode_data_path)
data_df['BMI'] = data_df['weight_kg'] / ((data_df['height_cm'] / 100) ** 2)
data_df['gender'] = label_encoder.fit_transform(data_df['gender'])
data_df['race'] = label_encoder.fit_transform(data_df['race'])
data_df['first_careunit'] = label_encoder.fit_transform(data_df['first_careunit'])
data_df['admission_type'] = label_encoder.fit_transform(data_df['admission_type'])
data_df['ventilator_mode_group_code'] = label_encoder.fit_transform(data_df['ventilator_mode_group'])
#data_df['ventilator_mode'] = label_encoder.fit_transform(data_df['ventilator_mode'])
data_df['insurance'] = label_encoder.fit_transform(data_df['insurance'])
data_df = data_df.drop(columns=['height_cm', 'weight_kg'])
data_df['RSBI'] =   data_df['resp_rate']/(data_df['tidal_volume_observed']* 0.001) 
data_df['minute_ventilation'] = data_df['tidal_volume_observed'] * data_df['resp_rate']* 0.001
print(data_df.columns)

Index(['subject_id', 'stay_id', 'hadm_id', 'charttime', 'before_weaning_hr',
       'age_now', 'gender', 'insurance', 'race', 'first_careunit',
       'admission_type', 'tobacco', 'RSBI', 'minute_ventilation',
       'ventilator_mode', 'ventilator_mode_group', 'peep', 'fio2',
       'tidal_volume_observed', 'respiratory_rate_set', 'plateau_pressure',
       'heart_rate', 'sbp', 'dbp', 'mbp', 'resp_rate', 'spo2', 'label', 'BMI',
       'ventilator_mode_group_code'],
      dtype='object')


In [164]:
unique_values_data_df = data_df['stay_id'].nunique()
unique_values_mode_df = mode_df['stay_id'].nunique()
unique_values_label_df = label_df['stay_id'].nunique()

print(unique_values_data_df, unique_values_mode_df, unique_values_label_df)

2663 2663 2840


In [165]:
def create_patient_group(label_df,mode_df):
    alive_list = [[],[],[],[],[],[]]
    dead_list = [[],[],[],[],[],[]]
    alive_num = [0,0,0,0,0]
    dead_num = [0,0,0,0,0]
    patient_set = set(mode_df['stay_id'])
    for index, row in flag_data_df.iterrows():
        if row['stay_id'] not in patient_set:
            continue
        id_mode_df = mode_df[mode_df['stay_id'] == row['stay_id']]
        count_complete_mode = id_mode_df['ventilator_mode_group'].tail(12).value_counts().get('Complete Support', 0)
        group_num = int(math.floor(count_complete_mode/3))
        label = id_mode_df['label'].iloc[0]
        if label == 1:
            alive_list[group_num].append(row['stay_id'])
            alive_num[group_num]+=1
        else:
            dead_list[group_num].append(row['stay_id'])
            dead_num[group_num]+=1
    return alive_list, dead_list, alive_num, dead_num

In [166]:
alive_list, dead_list, alive_num, dead_num = create_patient_group(label_df,mode_df)
print(alive_num)
print(dead_num)

[1187, 163, 127, 96, 119]
[520, 51, 47, 53, 300]


In [167]:
for i in range(5):
    print(dead_num[i]/(alive_num[i]+dead_num[i]))

0.30462800234329235
0.2383177570093458
0.27011494252873564
0.35570469798657717
0.7159904534606205


In [168]:
def split_list(input_list):
    total_length = len(input_list)
    part1_length = int(total_length * 0.7)
    part2_length = int(total_length * 0.2)
    part3_length = total_length - part1_length - part2_length
    part1 = random.sample(input_list, part1_length)
    remaining_list = [element for element in input_list if element not in part1]

    part2 = random.sample(remaining_list, part2_length)
    part3 = [element for element in remaining_list if element not in part2]

    return part1, part2, part3

In [ ]:
def create_data(alive_list, dead_list):
    train_data_id = []
    val_data_id = []
    test_data_id = []
    for i in range(5):
        train, val, test = split_list(alive_list)
        train_data_id.append(train)
        val_data_id.append(val)
        test_data_id.append(test)
        train, val, test = split_list(dead_list)
        train_data_id.append(train)
        val_data_id.append(val)
        test_data_id.append(test)
    